<h3>VVI</h3>

In [26]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain.agents import tool
from langchain.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
@tool
def getWordLength(word: str) -> int:
    """
        Returns the Length of a Word
    """
    return len(word)

In [6]:
getWordLength.invoke(input={"word":"iku"})

3

In [7]:
tools=[getWordLength]

In [11]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a very powerful assistant, but don't know current events"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [12]:
llmWithTools=llmGemini.bind_tools(tools=tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [15]:
agent={
        "input":lambda x: x['input'],
        "agent_scratchpad":lambda x: format_to_openai_tool_messages(x['intermediate_steps']) 
        }\
        |prompt\
        |llmWithTools\
        |OpenAIToolsAgentOutputParser()

In [17]:
agentExecutor=AgentExecutor(agent=agent, tools=tools,verbose=True)

In [18]:
agentExecutor.invoke(input={"input":"How many letters are there in `Eklavya`"})



> Entering new AgentExecutor chain...
{'name': 'getWordLength', 'description': 'Returns the Length of a Word', 'parameters': {'type_': 6, 'description': 'Returns the Length of a Word', 'properties': {'word': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['word'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}

Invoking: `getWordLength` with `{'word': 'Eklavya'}`


7{'name': 'getWordLength', 'description': 'Returns the Length of a Word', 'parameters': {'type_': 6, 'description': 'Returns the Length of a Word', 'properties': {'word': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['word'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
There are 7 letters in Eklavya. 


> Finished chain.


{'input': 'How many letters are there in `Eklavya`',
 'output': 'There are 7 letters in Eklavya. \n'}

<h3> Adding Memories </h3>

In [20]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a very powerful assistant, but don't know current events"),
        ("placeholder","{chat_history}"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [21]:
agent={
        "input":lambda x: x['input'],
        "chat_history": lambda x: x['chat_history'],
        "agent_scratchpad":lambda x: format_to_openai_tool_messages(x['intermediate_steps']) 
        }\
        |prompt\
        |llmWithTools\
        |OpenAIToolsAgentOutputParser()

In [22]:
agentExecutor=AgentExecutor(agent=agent, tools=tools,verbose=True)

In [27]:
messageHistory=ChatMessageHistory()

In [28]:
agentExecutorWithHistory=RunnableWithMessageHistory(
    get_session_history=lambda session_id:messageHistory,
    runnable=agentExecutor,
    input_messages_key="input",
    history_messages_key="chat_history",                   
)

In [29]:
session_id="<iku>"

In [30]:
agentExecutorWithHistory.invoke(
                                input={'input':"What is the total length for the word Tania?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
{'name': 'getWordLength', 'description': 'Returns the Length of a Word', 'parameters': {'type_': 6, 'description': 'Returns the Length of a Word', 'properties': {'word': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['word'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}

Invoking: `getWordLength` with `{'word': 'Tania'}`


5{'name': 'getWordLength', 'description': 'Returns the Length of a Word', 'parameters': {'type_': 6, 'description': 'Returns the Length of a Word', 'properties': {'word': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['word'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
The total length of the word Tania is 5. 


> Finished 

{'input': 'What is the total length for the word Tania?',
 'chat_history': [],
 'output': 'The total length of the word Tania is 5. \n'}

In [31]:
agentExecutorWithHistory.invoke(
                                input={'input':"What question did I ask?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
{'name': 'getWordLength', 'description': 'Returns the Length of a Word', 'parameters': {'type_': 6, 'description': 'Returns the Length of a Word', 'properties': {'word': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['word'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
You asked: "What is the total length for the word Tania?" 


> Finished chain.


{'input': 'What question did I ask?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The total length of the word Tania is 5. \n', additional_kwargs={}, response_metadata={})],
 'output': 'You asked: "What is the total length for the word Tania?" \n'}

In [32]:
agentExecutorWithHistory.invoke(
                                input={'input':"Is  this an Indian Name?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
{'name': 'getWordLength', 'description': 'Returns the Length of a Word', 'parameters': {'type_': 6, 'description': 'Returns the Length of a Word', 'properties': {'word': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['word'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
I do not have access to real-time information, including cultural or origin details about names.  Therefore, I cannot confirm if "Tania" is an Indian name. 


> Finished chain.


{'input': 'Is  this an Indian Name?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The total length of the word Tania is 5. \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What question did I ask?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You asked: "What is the total length for the word Tania?" \n', additional_kwargs={}, response_metadata={})],
 'output': 'I do not have access to real-time information, including cultural or origin details about names.  Therefore, I cannot confirm if "Tania" is an Indian name. \n'}

In [33]:
agentExecutorWithHistory.invoke(
                                input={'input':"Is this an English Word?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
{'name': 'getWordLength', 'description': 'Returns the Length of a Word', 'parameters': {'type_': 6, 'description': 'Returns the Length of a Word', 'properties': {'word': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['word'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
I do not have access to real-time information, including dictionaries or language databases. Therefore, I cannot confirm if "Tania" is an English word. 


> Finished chain.


{'input': 'Is this an English Word?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The total length of the word Tania is 5. \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What question did I ask?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You asked: "What is the total length for the word Tania?" \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Is  this an Indian Name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I do not have access to real-time information, including cultural or origin details about names.  Therefore, I cannot confirm if "Tania" is an Indian name. \n', additional_kwargs={}, response_metadata={})],
 'output': 'I do not have access to real-time information, including dictionaries or language databases. Therefore, I cannot confirm if "Tania" is an English word. \n'}